In [54]:
%reload_ext autoreload
%autoreload 2

In [55]:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY") 



In [56]:
chat = ChatOpenAI(
    openai_api_key=openai_key, 
    model='gpt-3.5-turbo'
)

In [57]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [58]:
res = chat(messages)

res

AIMessage(content="String theory is a theoretical framework in physics that aims to describe the fundamental nature of particles and forces in the universe. It suggests that at the most fundamental level, everything in the universe is made up of tiny, vibrating strings of energy.\n\nIn traditional particle physics, elementary particles like electrons and quarks are considered point-like particles with no internal structure. However, in string theory, these particles are replaced by tiny, one-dimensional strings. The vibrations of these strings give rise to different particles and their properties, such as mass and charge.\n\nOne of the key ideas in string theory is that it requires more than the usual four dimensions (three spatial dimensions and one time dimension) of spacetime. In fact, string theory predicts the existence of extra dimensions, which are compactified or curled up in such a way that they are not directly observable at our macroscopic scale.\n\nString theory also attemp

In [59]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to provide a unified theory for several reasons:

1. Consistency with quantum mechanics: String theory is inherently a quantum theory, meaning it describes particles and forces in a way consistent with the principles of quantum mechanics. This is important because quantum mechanics successfully describes the behavior of particles at the microscopic level. By incorporating quantum mechanics, string theory offers a framework that can potentially unify all fundamental forces, including gravity.

2. Gravity and the other forces: One of the main challenges in physics is reconciling gravity, described by Einstein's theory of general relativity, with the other three fundamental forces (electromagnetism, strong nuclear force, and weak nuclear force), which are described by quantum field theory. String theory provides a way to describe gravity in a quantum framework, potentially allowing for the unification of gravity with the other forces

loading the challenge document 

In [60]:
import PyPDF2
import os 

# Open the PDF file in binary read mode
with open("/home/hp/Documents/week 5/precision-RAG-prompt-tuning/prompt_generation/challenge.pdf",'rb') as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)

    num_pages = len(pdf_reader.pages)

    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        extracted_text = page_obj.extract_text()
        
len(extracted_text)


511

In [73]:
chunk_size=50
overlap=20
def chunk_text(extracted_text, chunk_size, overlap):
    chunks = []
    text_length = len(extracted_text)

    for start in range(0, text_length, chunk_size - overlap):
        end = min(start + chunk_size, text_length)
        chunk = extracted_text[start:end]
        chunks.append(chunk)

    return chunks

chunks = chunk_text(extracted_text, chunk_size, overlap)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:", chunk)

Chunk 1: References
●
Meistrari
didn’t
see
a
good
solution

Chunk 2: see
a
good
solution
for
prompt
engineering,
so
it’
Chunk 3: 
engineering,
so
it’s
building
one
●
AutoPrompt:
E
Chunk 4: 
one
●
AutoPrompt:
Eliciting
Knowledge
from
Langua
Chunk 5: nowledge
from
Language
Models
with
Automatically
G
Chunk 6: with
Automatically
Generated
Prompts
●
Large
Langu
Chunk 7: rompts
●
Large
Language
Models
Are
Human-Level
Pro
Chunk 8: 
Are
Human-Level
Prompt
Engineers
●
Prompt
Enginee
Chunk 9: ers
●
Prompt
Engineering
●
How
to
Create
a
Monte
C
Chunk 10: 
to
Create
a
Monte
Carlo
Simulation
using
Python
●
Chunk 11: ation
using
Python
●
Monte
Carlo
Method
Explained

Chunk 12: lo
Method
Explained
●
What
is
Monte
Carlo
Simulati
Chunk 13: Monte
Carlo
Simulation?
How
does
it
work?
●
Elo
Ra
Chunk 14: es
it
work?
●
Elo
Rating
Algorithm
●
Elo
algorithm
Chunk 15: ithm
●
Elo
algorithm
implementation
in
Python
●
Tr
Chunk 16: ation
in
Python
●
TrueSkillTM:
A
Bayesian
skill
ra
Chunk 17: 
A
Bayesian
skill
ratin

In [74]:
chunks = chunk_text(extracted_text, chunk_size, overlap)

#for i, chunk in enumerate(chunks):
    #print(f"Chunk {i + 1}:", chunk)
print(chunks[0])
len(chunks)

References
●
Meistrari
didn’t
see
a
good
solution



18

initialize the client and connect to the targetted index 

In [75]:
from pinecone import Pinecone

pc = Pinecone(api_key="cc6b9914-0016-4fa4-ab44-f82fe08434b9")
index = pc.Index("mekdes-index")

embed and add chunks to pinecone index 

In [76]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [77]:
res = embed_model.embed_documents(chunks)
len(res), len(res[0])


(18, 1536)

In [78]:
import time
import hashlib 

def generate_unique_id_for_chunk(chunk):
    # Generate a unique ID using a combination of timestamp and hash of the chunk content
    timestamp = str(time.time())  # Current timestamp
    chunk_hash = hashlib.sha256(chunk.encode()).hexdigest()  # Hash of the chunk content
    unique_id = f"{timestamp}_{chunk_hash}"
    
    return unique_id

In [79]:
# Iterate over chunks and corresponding embeddings to upsert into Pinecone
for chunk, embedding in zip(chunks, res):
    document_id = generate_unique_id_for_chunk(chunk) 
    index.upsert(vectors=[(document_id, embedding)])



In [80]:
# Retrieve the nearest neighbors of a vector
index_name = "mekdes-index"
query_vector = [0.1, 0.2, 0.3]  # Replace with your desired vector
top_k = 5  # Number of nearest neighbors to retrieve
import pinecone 
index = pinecone.Index(index_name, host='https://mekdes-index-nn3xpxm.svc.gcp-starter.pinecone.io')



In [81]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

In [82]:
def generate_prompt_from_vector(chunks, user_question):
    prompt = chunks + " \n based on the above data, give an answer to \
            the following question. restrict yourself to the above data only. \
            if you can't get an answer based on the data, you can feel free to \
                say i don't know. here is the question. \n" + user_question
    return prompt


In [83]:
import numpy as np

def cosine_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

In [84]:
query = "What is so special about Llama 2?"

cosine_similarity(query, embedding)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U33'), dtype('<U23')) -> None

In [ ]:
def generate_prompt(request):
    if request.method == 'POST':
        input_text = request.POST.get('input_text')
        # Embed the user's input
        embeded_question = embed_text([input_text])[0]
        highest_similarity = -1
        best_text_chunk = None
        # Compare with embeddings in TextChunk objects
        for text_chunk in chunks.objects.all():
            similarity = cosine_similarity(embeded_question, text_chunk.embed)
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_text_chunk = text_chunk.chunk
        if best_text_chunk is not None:
            generated_prompt = generate_prompt_from_vector(best_text_chunk, input_text)
            # return HttpResponse(generated_prompt)
            return render(request, 'prompt_app/prompt_result.html', {'generated_prompt': generated_prompt})
        else:
            return HttpResponse("No similar documents found.")
    else:
        return render(request, 'prompt_app/generate_prompt.html')